In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  2 17:13:01 2020

@author: mor
"""


import re
import nltk
import codecs
import string
import subprocess
import unicodedata
import os
from io import StringIO
import pandas as pd
from operator import itemgetter
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from gensim.models import Word2Vec,FastText
from gensim.models.doc2vec import Doc2Vec
from gensim.models.word2vec import LineSentence
import pickle
import time
import multiprocessing
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans



In [ ]:


BASE = os.getcwd()
DATA = os.path.join(BASE,'data')
MODELS = os.path.join(BASE,'models')



In [ ]:

# =============================================================================
# Evaluate word2vec, fastText and doc2vec
# =============================================================================


In [ ]:
# =============================================================================
# word2vec, fastText evaluate
# =============================================================================

# load models
modelWord2Vec =  Word2Vec.load(MODELS+"/word2vec.model")
modelFastText =  FastText.load(MODELS+"/fasttext.model")

In [ ]:
# =============================================================================
# model vectors
# =============================================================================
modelWord2Vec_hb=modelWord2Vec.wv
modelFastText_hb=modelFastText.wv


In [ ]:
# =============================================================================
# model corpus
# =============================================================================
words_Word2Vec=[]
for word in modelWord2Vec_hb.vocab:
    words_Word2Vec.append(word)

words_FastText=[]
for word in modelFastText_hb.vocab:
    words_FastText.append(word)


In [ ]:
  
# Printing out number of tokens available
print("Number of Tokens word2vec: {}".format(len(words_Word2Vec)))
print("Number of Tokens word2vec: {}".format(len(words_FastText)))

# Printing out the dimension of a word vector 
print("Dimension of a word vector: {}".format(len(modelWord2Vec_hb[words_Word2Vec[0]])))
print("Dimension of a word vector: {}".format(len(modelFastText_hb[words_FastText[0]])))


In [ ]:

# Pick a word 
#word1, word2, word3 .....
find_similar_to = 'word1'

# Finding out similar words [default= top 10]
top=20
for similar_word in modelWord2Vec_hb.similar_by_word(find_similar_to,top):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

print("\n")

for similar_word in modelFastText_hb.similar_by_word(find_similar_to,top):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))


In [ ]:
# =============================================================================
# similarity    
# =============================================================================
sim=modelFastText_hb.similarity('word1', 'word2')


In [ ]:
model = modelFastText_hb
words = words_FastText


In [ ]:

import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
#import random

#random.seed(100)
# Limit number of tokens to be visualized
limit = 600
vector_dim = 300

embedding = []
#randword=random.sample(words, limit)

# Appending the vectors 
for word in words[:limit]:
    embedding.append(model[word])


# Reshaping the embedding vector
embedding=np.array(embedding)
#embedding = embedding.reshape(limit, vector_dim)


In [ ]:

#plot tsne
def plot_with_labels(low_dim_embs, labels, filename='tsne.png',reverse=False):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    
    reverse_text = -1 if reverse else 1
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label[::reverse_text],##reverse text
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
#    plt.savefig(filename)
    plt.show()


In [ ]:

# Creating the tsne plot [Warning: will take time]
tsne = TSNE(perplexity=30.0, n_components=2, init='pca', n_iter=5000)

low_dim_embedding = tsne.fit_transform(embedding)

In [ ]:

# Finally plotting and saving the fig 
plot_with_labels(low_dim_embedding, words[:limit])


In [ ]:

# =============================================================================
# word embedding for documents/cases
# =============================================================================


In [ ]:

#tokenize
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\S+')

df = pd.read_csv('data/cases.csv')
df["tokens"] = df["text"].apply(tokenizer.tokenize)


In [ ]:
# =============================================================================
# average embedding
# =============================================================================
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=100):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, tokenDF, generate_missing=False):
    embeddings = tokenDF.apply(lambda x: get_average_word2vec(x, vectors, generate_missing=generate_missing))
    return list(embeddings)


In [ ]:
# =============================================================================
# calculations embedding for cases
# =============================================================================

embeddingAVG = get_word2vec_embeddings(model, df["tokens"])
embeddingAVG = np.array(embeddingAVG)

In [ ]:

caselabels = list(df.case)
limit=500


In [ ]:

from sklearn.manifold import TSNE

# Creating the tsne plot [Warning: will take time]
tsne = TSNE(perplexity=30.0, n_components=2, init='pca', n_iter=5000)

low_dim_embeddingAVG = tsne.fit_transform(embeddingAVG[:limit])


In [ ]:

# Finally plotting and saving the fig 
plot_with_labels(low_dim_embeddingAVG, caselabels)


In [ ]:

from sklearn.cluster import KMeans

ks = range(1, 50)
inertias = []

for k in ks:
    # Create a KMeans instance with k clusters: model
    modelKMeans=KMeans(n_clusters=k)
    
    # Fit model to samples
    modelKMeans.fit(embeddingAVG[:limit])
    
    # Append the inertia to the list of inertias
    inertias.append(modelKMeans.inertia_)
plt.figure(figsize=(12, 12))    
# Plot ks vs inertias
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
#plt.savefig('kmeans_inertia.png')
plt.show()


In [ ]:

# Import KMeans
from sklearn.cluster import KMeans

# Create a KMeans instance with 3 clusters: model
modelKMeans = KMeans(n_clusters=22)

# Fit model to points
modelKMeans.fit(embeddingAVG[:limit])

labels = modelKMeans.predict(embeddingAVG[:limit])


In [ ]:

# Import pyplot
import matplotlib.pyplot as plt

# Assign the columns of new_points: xs and ys
xs = low_dim_embeddingAVG[:,0]
ys = low_dim_embeddingAVG[:,1]
plt.figure(figsize=(24, 16))
# Make a scatter plot of xs and ys, using labels to define the colors
plt.scatter(xs,ys,c=labels,alpha=0.5)
for x, y, number in zip(xs, ys, range(0,len(xs))):
    plt.annotate(number, (x, y), xytext=(5, 5),textcoords='offset points',
                 ha='right',
                 va='bottom')
# Assign the cluster centers: centroids
centroids = modelKMeans.cluster_centers_

# Assign the columns of centroids: centroids_x, centroids_y
centroids_x = centroids[:,0]
centroids_y = centroids[:,1]

# Make a scatter plot of centroids_x and centroids_y
plt.scatter(centroids_x,centroids_y,marker='D',s=50,color='red')
plt.savefig('kmeansDoc.png')
plt.show()


In [ ]:
# =============================================================================
# simmilarity
# =============================================================================

indices = np.random.permutation(np.arange(len(embeddingAVG)))[:limit]
dfemg=pd.DataFrame(embeddingAVG[indices],index=indices).sort_index()


In [ ]:

simmilarity=dfemg.T.corr()
print(simmilarity[9].nsmallest(20))


In [ ]:

pd.DataFrame(low_dim_embeddingAVG).sort_index().T.corr()[9].nlargest(20) #.nsmallest(20)


In [ ]:

pd.DataFrame(embeddingAVG[[214,477]]).sort_index().T.corr()


In [ ]:

plt.style.use('ggplot')

# =============================================================================
# heat map for similarity
# =============================================================================

def heatMap(df, mirror, filename,annot=False):

    # Create Correlation df
    corr = df#.corr()
    # Plot figsize
    fig, ax = plt.subplots(figsize=(12, 12))
    # Generate Color Map
    colormap = sns.diverging_palette(220, 10, as_cmap=True)
   
    if mirror == True:
       #Generate Heat Map, allow annotations and place floats in map
       sns.heatmap(corr, cmap=colormap, annot=annot, fmt=".2f")
       #Apply xticks
#       plt.xticks(range(len(corr.columns)), corr.columns);
       #Apply yticks
#       plt.yticks(range(len(corr.columns)), corr.columns)
       #show plot

    else:
       # Drop self-correlations
       dropSelf = np.zeros_like(corr)
       dropSelf[np.triu_indices_from(dropSelf)] = True# Generate Color Map
       colormap = sns.diverging_palette(220, 10, as_cmap=True)
       # Generate Heat Map, allow annotations and place floats in map
       sns.heatmap(corr, cmap=colormap, fmt=".2f",annot=annot, mask=dropSelf)
       # Apply xticks
#       plt.xticks(range(len(corr.columns)), corr.columns);
       # Apply yticks
#       plt.yticks(range(len(corr.columns)), corr.columns)
    # show plot
    if filename:
        plt.savefig(filename)
    plt.show()


In [ ]:

heatMap(simmilarity,False,filename='heatmap2dim.png') 


In [ ]:
# =============================================================================
# Keyword Similarity - represent the relevancy of documnet by vetor of similarity values
# =============================================================================

#list of key words
litools=['kword1','kword2','kword3']


In [ ]:

def tokenssim(s,col):
    listtokens=list(s['newtokens'])
    sim=0
    simmax=0
    wordmax=''
    for i in listtokens:
        sim=modelFastText_hb.similarity(i, col)
        if sim>simmax:
            simmax=sim
            wordmax=i
    s[col+'0']=wordmax
    s[col+'1']=simmax
    return s


In [ ]:

for c in litools:
    df=df.apply(lambda row: tokenssim(row,col=c), axis=1)


In [ ]:

# =============================================================================
# doc2vec evaluate
# =============================================================================


In [ ]:

from gensim.models.doc2vec import Doc2Vec

#load model
model = Doc2Vec.load(MODELS+"/doc2vec.model")

df = pd.read_csv(DATA+'/cases.csv')


In [ ]:

#to find the vector of a document which is not in training data
test_data = word_tokenize("word1 word22 word12")
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('id')
print(similar_doc)


# to find vector of doc in training data using tags(id)
print(model.docvecs['id'])


In [ ]:

from sklearn.metrics.pairwise import cosine_similarity
vects=[]
columns=[]
for i,row in df.iterrows():
    vects.append(model.docvecs[str(row['id'])])
    columns.append(row['id'])


In [ ]:

vectsnp=np.array(vects)
columnsnp=np.array(columns)
dfvec=pd.DataFrame(vectsnp,index=columnsnp)


In [ ]:
# =============================================================================
# similarity
# =============================================================================

sim=cosine_similarity(dfvec)
cases=dfvec.index
simdf=pd.DataFrame(sim)


In [ ]:
 
heatMap(simdf,False)


In [ ]:

heatMap(simdf)


In [ ]:

heatMap(simdf.iloc[70:80,38:40],mirror=True,annot=True,filename='heatMapdocZoom.png')


In [ ]:

heatMap(simdf.iloc[70:80,0:10],mirror=True,annot=True,filename='heatMapdocZoom2.png')


In [ ]:
# =============================================================================
# check text cases
# =============================================================================

case1=cases[73]
case2=cases[39]

textdf=df[np.logical_or(df.case==case1,df.case==case2)]
for idx , row in textdf.iterrows():
    print(row['text'])
    print("\n\n")


In [ ]:

case1=cases[73]
case2=cases[3]

textdf=df[np.logical_or(df.case==case1,df.case==case2)]
for idx , row in textdf.iterrows():
    print(row['text'])
    print("\n\n")


In [ ]:
# =============================================================================
# find similar docs
# =============================================================================

#document len
textdf['ln']=textdf.text.str.len()



In [ ]:

threshold = 0.6

docslist=[]
similar_docs=set()
for i,row in textdf.iterrows():
    similar_doc,val = model.docvecs.most_similar(row['id'],topn=1)[0]
    if val > threshold:
        similar_docs.add(similar_doc)
        if row['id'] in similar_docs:
            continue
        print(similar_doc,val)
        docslist.append([row['id'],similar_doc,val,row['ln']])


In [ ]:

docdf=pd.DataFrame(docslist,columns=['doc_a','doc_b','sim','ln']).sort_values('sim',ascending=False)

print(docdf.head(10))


In [ ]:

docdf[np.logical_and(docdf['sim'].between(0.7,1),docdf['ln']>400)]


In [ ]:

docdf[docdf['sim'].between(0.7,0.91)]
